### Import Libraries, declare variables

In [1]:
import os
import re
import google
from google.oauth2 import credentials
from google.oauth2 import service_account
from google.oauth2.service_account import Credentials
from datetime import date
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import pandas as pd 
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder

# build model
import xgboost as xgb
from sklearn.metrics import roc_auc_score

SERVICE_TYPE = 'shs_invol_churn'
DATASET_ID = 'shs_invol_churn'
PROJECT_ID = 'divg-groovyhoon-pr-d2eab4' #mapping['PROJECT_ID']
RESOURCE_BUCKET = 'divg-groovyhoon-pr-d2eab4-default' #mapping['resources_bucket']
FILE_BUCKET = 'divg-groovyhoon-pr-d2eab4-default' #mapping['gcs_csv_bucket']
REGION = 'northamerica-northeast1' #mapping['REGION']
MODEL_ID = '9999'
FOLDER_NAME = 'telus_postpaid_churn'.format(MODEL_ID)
QUERIES_PATH = 'vertex_pipelines/' + FOLDER_NAME + '/queries/'
TABLE_ID = 'shs_invol_churn_data_final'

# scoringDate = date(2023, 9, 1)  # date.today() - relativedelta(days=2)- relativedelta(months=30)
# valScoringDate = date(2023, 11, 1)  # scoringDate - relativedelta(days=2)


### import bq to dataframe function

In [2]:
import pandas as pd 
import numpy as np 
from google.cloud import bigquery
from google.oauth2 import credentials

def import_bq_to_dataframe(project_id, dataset_id, table_id, client): 
    
    """
    Imports a specific table from BigQuery to a DataFrame. 
    
    Args: 
        project_id: The name of the project_id where the table is located.
        dataset_id: The name of the dataset_id where the table is located.
        table_id: The name of the table_id you wish to import to DataFrame.
        client: A BigQuery client instance. e.g. client = bigquery.Client(project=project_id).

    Returns: 
        A DataFrame
        
    Example: 
        import_bq_to_dataframe('bi-stg-divg-speech-pr-9d940b', 'call_to_retention_dataset', 'bq_ctr_pipeline_dataset')
        
    """
    
    sql = f"SELECT * FROM `{project_id}.{dataset_id}.{table_id}`"
    
    df_return = client.query(sql).to_dataframe()

    return df_return
    

### define get_lift function, import df_train and df_test from gcs bucket

In [3]:
import gc
import time
import pandas as pd
import numpy as np
import pickle
from google.cloud import storage
from google.cloud import bigquery
from sklearn.model_selection import train_test_split

project_id = PROJECT_ID
region = REGION
resource_bucket = RESOURCE_BUCKET
file_bucket = FILE_BUCKET
service_type=SERVICE_TYPE
project_id=PROJECT_ID
dataset_id=DATASET_ID
table_id = TABLE_ID

def get_lift(prob, y_test, q):
    result = pd.DataFrame(columns=['Prob', 'Churn'])
    result['Prob'] = prob
    result['Churn'] = y_test
    # result['Decile'] = pd.qcut(1-result['Prob'], 10, labels = False)
    result['Decile'] = pd.qcut(result['Prob'], q, labels=[i for i in range(q, 0, -1)])
    add = pd.DataFrame(result.groupby('Decile')['Churn'].mean()).reset_index()
    add.columns = ['Decile', 'avg_real_churn_rate']
    result = result.merge(add, on='Decile', how='left')
    result.sort_values('Decile', ascending=True, inplace=True)
    lg = pd.DataFrame(result.groupby('Decile')['Prob'].mean()).reset_index()
    lg.columns = ['Decile', 'avg_model_pred_churn_rate']
    lg.sort_values('Decile', ascending=False, inplace=True)
    lg['avg_churn_rate_total'] = result['Churn'].mean()
    lg = lg.merge(add, on='Decile', how='left')
    lg['lift'] = lg['avg_real_churn_rate'] / lg['avg_churn_rate_total']

    return lg

def get_gcp_bqclient(project_id, use_local_credential=True):
    token = os.popen('gcloud auth print-access-token').read()
    token = re.sub(f'\n$', '', token)
    credentials = google.oauth2.credentials.Credentials(token)

    bq_client = bigquery.Client(project=project_id)
    if use_local_credential:
        bq_client = bigquery.Client(project=project_id, credentials=credentials)
    return bq_client

client = get_gcp_bqclient(project_id)

df = import_bq_to_dataframe(project_id, dataset_id, table_id, client)

print(f'df: {df.shape}')


df: (796251, 189)


### add targets to df_train and df_target 

- df_target_train is from `divg-josh-pr-d1cc3a.tos_crosssell.bq_tos_cross_sell_targets_q3` 
- df_target_test is from `divg-josh-pr-d1cc3a.tos_crosssell.bq_tos_cross_sell_targets_q4` 
- some parts of the code and sql queries need to be dynamically adjusted to be included in the deploy model

In [4]:
# Define the start and end dates of the range
train_start_date = date(2023, 1, 1)
train_end_date = date(2023, 9, 30)

test_start_date = date(2023, 10, 1)
test_end_date = date(2024, 1, 1)

# Filter the DataFrame
df_train = df[(df['Base_Snapshot_Date'] >= train_start_date) & (df['Base_Snapshot_Date'] <= train_end_date)]
print(df_train.shape)

df_test = df[(df['Base_Snapshot_Date'] >= test_start_date) & (df['Base_Snapshot_Date'] <= test_end_date)]
print(df_test.shape)

#set up features (list)
cols_1 = df_train.columns.values
cols_2 = df_test.columns.values

cols = set(cols_1).intersection(set(cols_2))

features_to_exclude = ['CUST_ID', 'Bus_Billing_Account_Num', 'Bus_Prod_Instnc_Id', 
                       'Base_Snapshot_Date', 'Security_Origin', 'ACQUIRED_FROM', 
                       'Acquisition_Source', 'vol', 'invol', 'churn']

features = [f for f in cols if f not in features_to_exclude]

ban_train = df_train[['CUST_ID', 'Bus_Billing_Account_Num', 'Bus_Prod_Instnc_Id']]
X_train = df_train[features]
y_train = np.squeeze(df_train['vol'].values)

ban_test = df_test[['CUST_ID', 'Bus_Billing_Account_Num', 'Bus_Prod_Instnc_Id']]
X_test = df_test[features]
y_test = np.squeeze(df_test['vol'].values)


(496735, 189)
(299516, 189)


### preprocess

In [5]:
# Now we need to transform the features of the feature store.
def encode_categorical_features(df):
    # Get a list of all categorical columns
    cat_columns = df.select_dtypes(include=['object', 'category']).columns.tolist()

    # Encode each categorical column
    for col in cat_columns:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])
        
    return df
    
X_train = encode_categorical_features(X_train)
X_test = encode_categorical_features(X_test)

#set up features (list)
cols_1 = X_train.columns.values
cols_2 = X_test.columns.values

cols = set(cols_1).intersection(set(cols_2))

features = [f for f in cols if f not in features_to_exclude]

X_train = X_train[features] 
X_test = X_test[features] 

print(X_train.columns) 

/tmp/ipykernel_1840/4257940439.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = le.fit_transform(df[col])
/tmp/ipykernel_1840/4257940439.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = le.fit_transform(df[col])
/tmp/ipykernel_1840/4257940439.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_

Index(['HasT_Pre', 'tot_inv_amt_2', 'DIY_Activated', 'other_crdt_amt_3',
       'Customer_Category', 'Nho_Discount_Months', 'smhm_net_amt_2',
       'smhm_chrg_amt_2', 'HasKoodo', 'med_income',
       ...
       'hsic_crdt_amt_3', 'Video_', 'prov_cd', 'Delivery_Technology__Account_',
       'HasLWC', 'sing_crdt_amt_2', 'Tenure_Month_Groups__All_',
       'ttv_disc_amt_3', 'CREDIT_VALUE_CD__group_', 'other_disc_amt_l12m'],
      dtype='object', length=179)


### fit training data in xgboost classifier

In [6]:

# build model and fit in training data
import xgboost as xgb
from sklearn.metrics import roc_auc_score

xgb_model = xgb.XGBClassifier(
    learning_rate=0.05,
    n_estimators=250,
    max_depth=12,
    min_child_weight=1,
    gamma=0,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='binary:logistic',
    nthread=4,
    scale_pos_weight=1,
    seed=27
)

xgb_model.fit(X_train, y_train)
print('xgb training done')


xgb training done


### make predictions on X_test set, assign deciles to the predicted values, and save in df_test_exp

In [7]:
#predictions on X_test
y_pred = xgb_model.predict_proba(X_test)[:, 1]

#join ban_test, X_test, y_test and y_pred and print to csv
#CHECK THE SIZE OF EACH COMPONENT BEFORE JOINING
q=10
df_ban_test = ban_test
df_test_exp = df_ban_test.join(X_test) 
df_test_exp['y_test'] = y_test
df_test_exp['y_pred_proba'] = y_pred
df_test_exp['y_pred'] = (df_test_exp['y_pred_proba'] > 0.5).astype(int)
df_test_exp['decile'] = pd.qcut(df_test_exp['y_pred_proba'], q, labels=[i for i in range(q, 0, -1)])

lg = get_lift(y_pred, y_test, q)

lg

/tmp/ipykernel_1840/4227574850.py:25: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  add = pd.DataFrame(result.groupby('Decile')['Churn'].mean()).reset_index()
/tmp/ipykernel_1840/4227574850.py:29: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  lg = pd.DataFrame(result.groupby('Decile')['Prob'].mean()).reset_index()


,Decile,avg_model_pred_churn_rate,avg_churn_rate_total,avg_real_churn_rate,lift
0,1,0.043349,0.01043,0.033555,3.217094
1,2,0.010612,0.01043,0.017895,1.715726
2,3,0.006531,0.01043,0.011552,1.107540
3,4,0.004548,0.01043,0.009415,0.902707
4,5,0.003365,0.01043,0.007812,0.749030
5,6,0.002563,0.01043,0.006544,0.627413
6,7,0.001970,0.01043,0.006243,0.598584
7,8,0.001494,0.01043,0.004841,0.464158
8,9,0.001079,0.01043,0.003472,0.332902
9,10,0.000631,0.01043,0.002971,0.284887


In [8]:
# Get feature importances from xgboost model
importances = xgb_model.feature_importances_

# Get the index of importances from greatest importance to least
sorted_index = np.argsort(importances)[::-1]
x = range(len(importances))

feature_names = X_train.columns

# Create tick labels 
labels = np.array(feature_names)[sorted_index]

for idx in sorted_index: 
    print(f'{feature_names[idx]}, {importances[idx]}', end='\n')

HSLTE_TOTAL, 0.029510151594877243
prov_cd, 0.027740316465497017
smhm_chrg_amt_l12m, 0.020153893157839775
Province_Grp, 0.0154129508882761
TV_Count, 0.013436856679618359
HasIPTV, 0.013394410721957684
TOS_FLAG, 0.012123427353799343
HS_CNT, 0.01190012227743864
HasTOS, 0.011616945266723633
Security_Plan_Group, 0.011611436493694782
IPTV_CNT, 0.010678873397409916
sing_crdt_amt_1, 0.01008572056889534
sing_net_amt_3, 0.009159926325082779
DIY_Activated, 0.008697567507624626
Price_Plan_Txt__SHS_, 0.008210493251681328
PRICE_PLAN_RATE_AMT__Custom_SQL_Query_, 0.008189907297492027
Channel, 0.008167806081473827
HSIA_Per_Cust, 0.008036294020712376
Installation_Type, 0.007838568650186062
tot_chrg_amt_l12m, 0.007722578477114439
smhm_net_amt_l12m, 0.007616798393428326
CHNL_ORG_TXT__Custom_SQL_Query_, 0.007614592090249062
HP_CNT, 0.007477073464542627
bi_chnl_tag_cd, 0.007379056420177221
tot_chrg_amt_1, 0.007371733896434307
tot_chrg_amt_2, 0.007369871251285076
tot_disc_amt_2, 0.007224651984870434
other_chr

### get feature importances from xgboost model

In [9]:
# Get feature importances from xgboost model
importances = xgb_model.feature_importances_

# Get the index of importances from greatest importance to least
sorted_index = np.argsort(importances)[::-1]
x = range(len(importances))

feature_names = X_train.columns

# Create tick labels 
labels = np.array(feature_names)[sorted_index]

In [10]:
labels

array(['HSLTE_TOTAL', 'prov_cd', 'smhm_chrg_amt_l12m', 'Province_Grp',
       'TV_Count', 'HasIPTV', 'TOS_FLAG', 'HS_CNT', 'HasTOS',
       'Security_Plan_Group', 'IPTV_CNT', 'sing_crdt_amt_1',
       'sing_net_amt_3', 'DIY_Activated', 'Price_Plan_Txt__SHS_',
       'PRICE_PLAN_RATE_AMT__Custom_SQL_Query_', 'Channel',
       'HSIA_Per_Cust', 'Installation_Type', 'tot_chrg_amt_l12m',
       'smhm_net_amt_l12m', 'CHNL_ORG_TXT__Custom_SQL_Query_', 'HP_CNT',
       'bi_chnl_tag_cd', 'tot_chrg_amt_1', 'tot_chrg_amt_2',
       'tot_disc_amt_2', 'other_chrg_amt_l12m', 'tot_tax_inv_amt_l12m',
       'Contract_Expiring_Soon', 'price_plan', 'hsic_net_amt_1',
       'tot_tax_inv_amt_2', 'smhm_disc_amt_2',
       'PRICE_PLAN_TXT__Custom_SQL_Query_', 'Security_Plan_Type',
       'Tenure_Month_Groups__All_', 'POSTAL_CD__Custom_SQL_Query_',
       'other_disc_amt_l12m', 'sing_net_amt_1', 'HasT_Post',
       'tot_tax_inv_amt_3', 'census_subdivision_desc', 'smhm_net_amt_3',
       'tot_inv_amt_1', 'tot

In [11]:
importances

array([0.        , 0.00596754, 0.00869757, 0.00410739, 0.00628898,
       0.00173986, 0.00601527, 0.00618127, 0.00579987, 0.00611591,
       0.00469603, 0.00573357, 0.00515294, 0.00324942, 0.006069  ,
       0.0027117 , 0.00143199, 0.01008572, 0.00563667, 0.00581843,
       0.00613164, 0.0064081 , 0.00557127, 0.00625598, 0.00482202,
       0.00382238, 0.00462583, 0.        , 0.00697473, 0.00651461,
       0.00488588, 0.0058676 , 0.00404467, 0.00605509, 0.00672026,
       0.0060973 , 0.00611667, 0.00674763, 0.00595321, 0.00576685,
       0.00581174, 0.00512827, 0.00722465, 0.00641813, 0.00595514,
       0.00619702, 0.        , 0.00598802, 0.00594651, 0.00593138,
       0.        , 0.00528319, 0.00625268, 0.00661796, 0.00563273,
       0.00554546, 0.00503723, 0.00566693, 0.00396056, 0.00660363,
       0.00477743, 0.02951015, 0.01161695, 0.00605452, 0.00645014,
       0.00515991, 0.        , 0.00478615, 0.0076168 , 0.00639398,
       0.00143335, 0.00449522, 0.00772258, 0.00642704, 0.00522

In [12]:
np.argsort(importances)[::-1]

array([ 61, 171,  83,  91, 100, 106, 144, 155,  62, 105, 122,  17, 117,
         2, 113,  90, 141, 148, 104,  72,  68,  82, 147,  80, 135, 133,
        42, 124,  86,  28, 123, 127, 168,  37,  78,  34, 175,  53, 178,
        59,  96, 131,  29, 108, 103,  79,  64,  73,  89,  43,  88,  21,
        69, 132, 170, 111,   4, 125,  95,  23, 150,  52, 116,  93, 107,
        45,  85,   7, 159,  77,  20,  36,   9,  35, 143, 134,  14,  33,
        63, 146,   6, 151,  47,   1,  44,  38,  48,  81,  49, 160, 166,
       149,  31,  19,  40,   8,  99, 164,  39,  11,  57,  18,  54, 121,
        76,  22, 163,  92, 129,  55, 130, 177, 153, 140, 167,  51, 152,
        74,  87,  65, 162,  12,  41,  56, 101, 157,  30,  24, 110,  67,
        60,  10,  26,  71, 136,   3,  32, 137,  58,  25, 176,  98,  13,
       158,  75, 169,  15, 126, 128,   5, 115,  70,  16, 142, 109, 114,
       174, 120, 119, 118, 173, 138, 112, 165,  66,  50, 161,  46, 156,
       154,  84,  94,  97, 102, 145, 172,  27, 139,   0])

In [13]:
error

NameError: name 'error' is not defined

### Load results to BQ - WIP

In [ ]:
SCORE_TABLE_ID = 'bq_telus_12_months_churn_scores'

project_id = PROJECT_ID 
dataset_id = DATASET_ID
score_table_id = SCORE_TABLE_ID

# get full score to cave into bucket
pred_prob = xgb_model.predict_proba(X_val, ntree_limit=xgb_model.best_iteration)[:, 1]
result = pd.DataFrame(columns=['ban', 'subscriber_no', 'score_date', 'y_true', 'y_pred'])

result['ban'] = list(ban_val['ban'])
result['ban'] = result['ban'].astype('str')

result['subscriber_no'] = list(ban_val['ban'])
result['subscriber_no'] = result['subscriber_no'].astype('str')

result['score_date'] = "2023-11-04"

result['y_true'] = list(y_val)
result['y_true'] = result['y_true'].fillna(0.0).astype('float64')

result['y_pred'] = list(pred_prob)
result['y_pred'] = result['y_pred'].fillna(0.0).astype('float64')

############# updated up to here ############

result.to_csv('gs://{}/{}/ucar/{}_prediction_v2.csv'.format(file_bucket, service_type, service_type), index=False)

# define dtype_bq_mapping
dtype_bq_mapping = {np.dtype('int64'): 'INTEGER', 
np.dtype('float64'):  'FLOAT', 
np.dtype('float32'):  'FLOAT', 
np.dtype('object'):  'STRING', 
np.dtype('bool'):  'BOOLEAN', 
np.dtype('datetime64[ns]'):  'DATE', 
pd.Int64Dtype(): 'INTEGER'} 

# export df_final to bigquery 
schema_list = [] 
for column in result.columns: 
    schema_list.append(bigquery.SchemaField(column, dtype_bq_mapping[result.dtypes[column]], mode='NULLABLE')) 
print(schema_list) 

dest_table = f'{project_id}.{dataset_id}.{score_table_id}'

# Sending to bigquery 
client = get_gcp_bqclient(project_id)
job_config = bigquery.LoadJobConfig(schema=schema_list, write_disposition='WRITE_TRUNCATE') 
job = client.load_table_from_dataframe(result, dest_table, job_config=job_config) 
job.result() 

table = client.get_table(dest_table) # Make an API request 
print("Loaded {} rows and {} columns to {}".format(table.num_rows, len(table.schema), table_id)) 

time.sleep(60)

# table_ref = f'{project_id}.{dataset_id}.{score_table}'
# client = bigquery.Client(project=project_id)
# table = client.get_table(table_ref)
# schema = table.schema

# ll = []
# for item in schema:
#     col = item.name
#     d_type = item.field_type
#     if 'float' in str(d_type).lower():
#         d_type = 'FLOAT64'
#     ll.append((col, d_type))

#     if 'integer' in str(d_type).lower():
#         result[col] = result[col].fillna(0).astype(int)
#     if 'float' in str(d_type).lower():
#         result[col] = result[col].fillna(0.0).astype(float)
#     if 'string' in str(d_type).lower():
#         result[col] = result[col].fillna('').astype(str)

# table_ref = '{}.{}.{}'.format(project_id, dataset_id, temp_table)
# client = bigquery.Client(project=project_id)
# if if_tbl_exists(client, table_ref):
#     client.delete_table(table_ref)

# client.create_table(table_ref)
# config = bigquery.LoadJobConfig(schema=schema)
# config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
# bq_table_instance = client.load_table_from_dataframe(result, table_ref, job_config=config)
# time.sleep(5)

# drop_sql = f"""delete from `{project_id}.{dataset_id}.{score_table}` where score_date = '{score_date_dash}'"""  # .format(project_id, dataset_id, score_date_dash)
# client.query(drop_sql)
# #
# load_sql = f"""insert into `{project_id}.{dataset_id}.{score_table}`
#               select * from `{project_id}.{dataset_id}.{temp_table}`"""
# client.query(load_sql)


In [ ]:
error

### save the model in gcs

In [ ]:
# save the model in GCS
from datetime import datetime
models_dict = {}
create_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
models_dict['create_time'] = create_time
models_dict['model'] = xgb_model
models_dict['features'] = features

with open('model_dict.pkl', 'wb') as handle:
    pickle.dump(models_dict, handle)
handle.close()

storage_client = storage.Client()
bucket = storage_client.get_bucket(file_bucket)

MODEL_PATH = '{}_xgb_models/'.format(service_type)
blob = bucket.blob(MODEL_PATH)
if not blob.exists(storage_client):
    blob.upload_from_string('')

model_name_onbkt = '{}{}_models_xgb_{}'.format(MODEL_PATH, service_type, models_dict['create_time'])
blob = bucket.blob(model_name_onbkt)
blob.upload_from_filename('model_dict.pkl')

print(f"....model loaded to GCS done at {str(create_time)}")

time.sleep(300)

### load the latest saved xgb_model to the environment

In [ ]:
# MODEL_PATH = '{}_xgb_models/'.format(service_type)
# df_score = pd.read_csv('gs://{}/{}_score.csv.gz'.format(file_bucket, service_type), compression='gzip')
# df_score.dropna(subset=['ban'], inplace=True)
# df_score.reset_index(drop=True, inplace=True)
# print('......scoring data loaded:{}'.format(df_score.shape))
# time.sleep(10)
from google.cloud import bigquery
from google.cloud import storage

MODEL_PATH = '{}_xgb_models/'.format(service_type)

def load_model(file_bucket: str, service_type: str): 
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(file_bucket)
    blobs = storage_client.list_blobs(file_bucket, prefix='{}{}_models_xgb_'.format(MODEL_PATH, service_type))

    model_lists = []
    for blob in blobs:
        model_lists.append(blob.name)

    blob = bucket.blob(model_lists[-1])
    blob_in = blob.download_as_string()
    model_dict = pickle.loads(blob_in)
    xgb_model = model_dict['model']
    features = model_dict['features']
    print('...... model loaded')
    time.sleep(10)
    
    return xgb_model, features

xgb_model, features = load_model(file_bucket = FILE_BUCKET, service_type = SERVICE_TYPE) 

### backup codes 

In [ ]:
def get_gcp_bqclient(project_id, use_local_credential=True):
    token = os.popen('gcloud auth print-access-token').read()
    token = re.sub(f'\n$', '', token)
    credentials = google.oauth2.credentials.Credentials(token)

    bq_client = bigquery.Client(project=project_id)
    if use_local_credential:
        bq_client = bigquery.Client(project=project_id, credentials=credentials)
    return bq_client

client = get_gcp_bqclient(project_id)

#instantiate df_target_train and df_target_test
sql_train = ''' SELECT * FROM `{}.{}.bq_tos_cross_sell_targets_q3` '''.format(project_id, dataset_id)
df_target_train = client.query(sql_train).to_dataframe()
df_target_train = df_target_train.loc[
    df_target_train['YEAR_MONTH'] == "2022-Q3"] #'-'.join(score_date_dash.split('-')[:2])]  # score_date_dash = '2022-08-31'

#set up df_train and df_test (add 'target')
df_target_train['ban'] = df_target_train['ban'].astype('int64')
df_target_train = df_target_train.groupby('ban').tail(1)

df_train = df_train.merge(df_target_train[['ban', 'product_crosssell_ind']], on='ban', how='left')
df_train.rename(columns={'product_crosssell_ind': 'target'}, inplace=True)
df_train.dropna(subset=['target'], inplace=True)
df_train['target'] = df_train['target'].astype(int)

df_train

In [ ]:
sum(df_train['target'])

In [ ]:
def get_gcp_bqclient(project_id, use_local_credential=True):
    token = os.popen('gcloud auth print-access-token').read()
    token = re.sub(f'\n$', '', token)
    credentials = google.oauth2.credentials.Credentials(token)

    bq_client = bigquery.Client(project=project_id)
    if use_local_credential:
        bq_client = bigquery.Client(project=project_id, credentials=credentials)
    return bq_client

client = get_gcp_bqclient(project_id)
sql_test = ''' SELECT * FROM `{}.{}.bq_tos_cross_sell_targets_q4` '''.format(project_id, dataset_id)
df_target_test = client.query(sql_test).to_dataframe()
df_target_test = df_target_test.loc[
    df_target_test['YEAR_MONTH'] == "2022-Q4"] #'-'.join(score_date_val_dash.split('-')[:2])]  # score_date_dash = '2022-09-30'

#set up df_train and df_test (add 'target')
df_target_test['ban'] = df_target_test['ban'].astype('int64')
df_target_test = df_target_test.groupby('ban').tail(1)

df_test = df_test.merge(df_target_test[['ban', 'product_crosssell_ind']], on='ban', how='left')
df_test.rename(columns={'product_crosssell_ind': 'target'}, inplace=True)
df_test.dropna(subset=['target'], inplace=True)
df_test['target'] = df_test['target'].astype(int)

df_test

In [ ]:
sum(df_test['target'])